### 腾讯广告转化率比赛
数据百度云地址，链接: https://pan.baidu.com/s/1kVduN1X 密码: nej7

### 基于AD统计的版本

CVR (Conversion Rate): 转化率。是一个衡量CPA广告效果的指标，CVR=(转化量/点击量)*100%。

In [5]:
import numpy as np
import pandas as pd

In [11]:
# load data
ad = pd.read_csv('../data/pre/ad.csv')
raw_train = pd.read_csv('../data/pre/train.csv')
raw_test = pd.read_csv('../data/pre/test.csv')

In [7]:
train.head()

,label,clickTime,conversionTime,creativeID,userID,positionID,connectionType,telecomsOperator
0,0,170000,NaN,3089,2798058,293,1,1
1,0,170000,NaN,1259,463234,6161,1,2
2,0,170000,NaN,4465,1857485,7434,4,1
3,0,170000,NaN,1004,2038823,977,1,1
4,0,170000,NaN,1887,2015141,3688,1,1


In [8]:
test.head()

,instanceID,label,clickTime,creativeID,userID,positionID,connectionType,telecomsOperator
0,1,-1,310000,3745,1164848,3451,1,3
1,2,-1,310000,2284,2127247,1613,1,3
2,3,-1,310000,1456,2769125,5510,2,1
3,4,-1,310000,4565,9762,4113,2,3
4,5,-1,310000,49,2513636,3615,1,3


In [10]:
ad.head()

,creativeID,adID,camgaignID,advertiserID,appID,appPlatform
0,4079,2318,147,80,14,2
1,4565,3593,632,3,465,1
2,3170,1593,205,54,389,1
3,6566,2390,205,54,389,1
4,5187,411,564,3,465,1


In [12]:
# process data
train = pd.merge(raw_train, ad, on='creativeID') #creativeID 素材ID，一个广告下可以有多个素材
test = pd.merge(raw_test, ad, on='creativeID')

# model building
key = 'appID' # 广告推广的目标页面链接地址
df_cvr = train.groupby(key).apply(lambda df: np.mean(df['label'])).reset_index()
df_cvr.columns = [key, 'avg_cvr']
test = pd.merge(test, df_cvr, how="left", on=key)

test['avg_cvr'].fillna(np.mean(train['label']), inplace=True)

In [29]:
# submission
# instanceID唯一标识一个样本，-1代表label占位使用，表示待预测
df = pd.DataFrame({'instanceID': test['instanceID'].values, 'proba': test['avg_cvr'].values})
df.sort_values('instanceID', inplace=True)
df.to_csv('submission.csv', index=False)